Type Hints Overview

* typing module in standard library since 3.5

In [5]:
import typing

In [20]:
from typeguard import typechecked

In [21]:
def greeting(name: str) -> str:
    return 'Hello ' + name

PEP literature:
    
* PEP 526: Syntax for Variable Annotations
* PEP 544: Protocols: Structural subtyping (static duck typing)
* PEP 585: Type Hinting Generics In Standard Collections
* PEP 586: Literal Types
* PEP 589: TypedDict: Type Hints for Dictionaries with a Fixed Set of Keys
* PEP 591: Adding a final qualifier to typing
* PEP 593: Flexible function and variable annotations
* PEP 604: Allow writing union types as X | Y
* PEP 612: Parameter Specification Variables
* PEP 613: Explicit Type Aliases
* PEP 646: Variadic Generics
* PEP 647: User-Defined Type Guards
* PEP 655: Marking individual TypedDict items as required or potentially missing
* PEP 673: Self type
* PEP 675: Arbitrary Literal String Type
* PEP 681: Data Class Transforms


Some concepts introduced by these PEPs:

* Literal, LiteralString, TypedDict, Required, NotRequired, Final, ... 
* Protocols and structural subtyping ("static duck typing")

Notes:

* Optional, checked statically or at runtime.
* anything not annotated is "Any" (escape hatch)
* typing is gradual, annotated regions and non-annotated regions can exist side by side 
* tooling support: mypy, pyre (pysa), ...

Pragmatics:
    
* documentation
* start with most used code or most critical code (and maybe leave it at that)


We can use variable annotations:
    
* functions arguments
* variables
* concrete types, generic classes



### Contents

* Special typing primitives (Any, Union, Optional, NoReturn, ...)
* Generic concrete collections (List, Dict, ...)
* Abstract base classes (~ collections.abc)


### Examples

#### Get type information about an object

In [22]:
def hello(name: str) -> str:
    return f"hello {name}"

In [23]:
import os

In [24]:
typing.get_type_hints(hello)

{'name': str, 'return': str}

In [25]:
hello.__annotations__

{'name': str, 'return': str}

#### Special directives

* alternative to other projects, like Pydantic
* alternative to other constraints (e.g. via descriptors)
* use case: data validation

In [26]:
class Point2D(typing.TypedDict):
    x: int
    y: int
    label: str

a: Point2D = {'x': 1, 'y': 2, 'label': 'good'}  # OK
b: Point2D = {'z': 3, 'label': 'bad'}           # Fails type check

assert Point2D(x=1, y=2, label='first') == dict(x=1, y=2, label='first')



In [27]:
class Point2D(typing.TypedDict, total=False):
    x: typing.Required[int]
    y: typing.Required[int]
    label: str

# Alternative syntax
Point2D = typing.TypedDict('Point2D', {
    'x': typing.Required[int],
    'y': typing.Required[int],
    'label': str
}, total=False)

#### Structural subtyping

> PEP 544 allows to solve this problem by allowing users to write the above code without explicit base classes in the class definition, allowing Bucket to be implicitly considered a subtype of both Sized and Iterable[int] by static type checkers. This is known as structural subtyping (or static duck-typing)

In [28]:
from collections.abc import Iterator, Iterable

class Bucket:  # Note: no base classes
    def __len__(self) -> int: return 0
    def __iter__(self) -> typing.Iterator[int]:
        return iter([])

def collect(items: typing.Iterable[int]) -> int:
    return 0

result = collect(Bucket())  # Passes type check

#### Type Aliases

In [29]:
Vector = typing.List[float] # list[float] since 3.9 ...

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

# passes type checking; a list of floats qualifies as a Vector.
new_vector = scale(2.0, [1.0, -4.2, 5.4])

In [30]:
!python --version

Python 3.11.0


#### Abstract Base classes

In [32]:
@typechecked
def infinite_stream(start: int) -> typing.Iterator[int]:
    while True:
        yield start
        start += 1

In [36]:
result = infinite_stream(3)

In [41]:
result

In [44]:
result